In [5]:
import numpy as np

def sigmoid(x):
  # Sigmoid activation function: f(x) = 1 / (1 + e^(-x))
  return 1 / (1 + np.exp(-x))

def deriv_sigmoid(x):
  # Derivative of sigmoid: f'(x) = e^(-x) / (1 + e^(-x))^2 = f(x) * (1 - f(x))
  fx = sigmoid(x)
  return fx*(1 - fx)

def mse(y_true, y_pred):
  # Loss function: mean squared error (MSE)  
  # y_true and y_pred are numpy arrays of the same length.
  return ((y_true - y_pred)**2).mean()

class Network:
  '''
  A very simple neural network with:
    - 2 inputs (x[0], x[1])
    - a hidden layer with 2 neurons (h1, h2)
    - an output layer with 1 neuron (o1)
  '''
  def __init__(self):
    # Weights and Biases are randomly initialized.
    
    # Weights (there are 6 weights for 3 neurons taking 2 inputs)
    self.w1 = np.random.normal()
    self.w2 = np.random.normal()
    self.w3 = np.random.normal()
    self.w4 = np.random.normal()
    self.w5 = np.random.normal()
    self.w6 = np.random.normal()

    # Biases (there are 3 biases = 1 bias per neuron)
    self.b1 = np.random.normal()
    self.b2 = np.random.normal()
    self.b3 = np.random.normal()

  def feedforward(self, x):
    # x is a numpy array with 2 elements.
    h1 = sigmoid(self.w1 * x[0] + self.w2 * x[1] + self.b1)
    h2 = sigmoid(self.w3 * x[0] + self.w4 * x[1] + self.b2)
    o1 = sigmoid(self.w5 * h1 + self.w6 * h2 + self.b3)
    return o1

  def train(self, data, all_y_trues):
    '''
    - data is a (n x 2) numpy array where n = number of samples in the dataset.
    - all_y_trues is a numpy array with n elements representing the gender (1 = woman, 0 = man).
    '''
    learn_rate = 0.1
    epochs = 1000 # number of times to loop through the entire dataset

    for epoch in range(epochs):
      for x, y_true in zip(data, all_y_trues):
        # FEEDFORWARD
        sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
        h1 = sigmoid(sum_h1)

        sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
        h2 = sigmoid(sum_h2)

        sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
        o1 = sigmoid(sum_o1)
        y_pred = o1

        # BACKPROPAGATION
        
        # Calculate partial derivatives.
        # Convention: d_L_d_w1 represents "partial L / partial w1"
        d_L_d_ypred = -2 * (y_true - y_pred)

        # Neuron o1
        d_ypred_d_w5 = h1 * deriv_sigmoid(sum_o1)
        d_ypred_d_w6 = h2 * deriv_sigmoid(sum_o1)
        d_ypred_d_b3 = deriv_sigmoid(sum_o1)

        d_ypred_d_h1 = self.w5 * deriv_sigmoid(sum_o1)
        d_ypred_d_h2 = self.w6 * deriv_sigmoid(sum_o1)

        # Neuron h1
        d_h1_d_w1 = x[0] * deriv_sigmoid(sum_h1)
        d_h1_d_w2 = x[1] * deriv_sigmoid(sum_h1)
        d_h1_d_b1 = deriv_sigmoid(sum_h1)

        # Neuron h2
        d_h2_d_w3 = x[0] * deriv_sigmoid(sum_h2)
        d_h2_d_w4 = x[1] * deriv_sigmoid(sum_h2)
        d_h2_d_b2 = deriv_sigmoid(sum_h2)

        # Update weights and biases
        # Neuron h1
        self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
        self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
        self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1

        # Neuron h2
        self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
        self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
        self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2

        # Neuron o1
        self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_w5
        self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w6
        self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3

      # Calculate total loss at the end of each epoch
      if epoch % 10 == 0:
        y_preds = np.apply_along_axis(self.feedforward, 1, data)
        loss = mse(all_y_trues, y_preds)
        print("Epoch %d loss: %.3f" % (epoch, loss))

# Define dataset

raw_data = np.array([ # showing height (cm) and weight (kg)
  [165, 55],  # Alice
  [180, 85],  # Bob
  [175, 77],  # Charlie
  [157, 52],  # Diana
])
print("original data:")
print(raw_data)
print("")
print("mean height and weigth: ", np.mean(raw_data, axis=0))
# To make things easier for the network our data are scaled on the mean values (of heigth and weight)
data = np.array(raw_data - np.mean(raw_data, axis=0))
print("")
print("rescaled data:")
print(data)
all_y_trues = np.array([ # 1 = woman, 0 = man
  1, # Alice
  0, # Bob
  0, # Charlie
  1, # Diana
])
print("")
# Train the neural network
network = Network()
network.train(data, all_y_trues)

original data:
[[165  55]
 [180  85]
 [175  77]
 [157  52]]

mean height and weigth:  [169.25  67.25]

rescaled data:
[[ -4.25 -12.25]
 [ 10.75  17.75]
 [  5.75   9.75]
 [-12.25 -15.25]]

Epoch 0 loss: 0.275
Epoch 10 loss: 0.184
Epoch 20 loss: 0.113
Epoch 30 loss: 0.072
Epoch 40 loss: 0.050
Epoch 50 loss: 0.038
Epoch 60 loss: 0.031
Epoch 70 loss: 0.027
Epoch 80 loss: 0.024
Epoch 90 loss: 0.022
Epoch 100 loss: 0.020
Epoch 110 loss: 0.018
Epoch 120 loss: 0.017
Epoch 130 loss: 0.016
Epoch 140 loss: 0.015
Epoch 150 loss: 0.014
Epoch 160 loss: 0.013
Epoch 170 loss: 0.013
Epoch 180 loss: 0.012
Epoch 190 loss: 0.011
Epoch 200 loss: 0.011
Epoch 210 loss: 0.010
Epoch 220 loss: 0.010
Epoch 230 loss: 0.010
Epoch 240 loss: 0.009
Epoch 250 loss: 0.009
Epoch 260 loss: 0.008
Epoch 270 loss: 0.008
Epoch 280 loss: 0.008
Epoch 290 loss: 0.008
Epoch 300 loss: 0.007
Epoch 310 loss: 0.007
Epoch 320 loss: 0.007
Epoch 330 loss: 0.007
Epoch 340 loss: 0.007
Epoch 350 loss: 0.006
Epoch 360 loss: 0.006
Epoch 370

In [6]:
# Make some predictions
emily = np.array([-9.25, -12.25]) # 160 cm, 55 kg
frank = np.array([15.75, 22.75])  # 185 cm, 90 kg
print("Emily: %.3f" % network.feedforward(emily)) # ~0.95 - F
print("Frank: %.3f" % network.feedforward(frank)) # ~0.04 - M

Emily: 0.946
Frank: 0.038
